# 서울시 공원 시각화

In [12]:
import folium
import pandas as pd
import re

In [3]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


In [186]:
park = pd.read_csv(filename, encoding='EUC-KR')
park_df = pd.DataFrame(park)
park_df.tail(3)

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
129,130,경춘선숲길,일제 강점기때 우리 민족의 자본으로 만든 최초의 철도시설로 1939년 7월 25일 ...,211392㎡,NaN,조경시설 : 텃밭 잔디광장 철길 산책로 등 건축시설 : 무궁화호 객차(방문자센터) ...,NaN,NaN,지하철 이용시 지하철 7호선 하계역 4번출구 미성/장미아파트 도로 450m직진 지하...,○ 6.3km (시 작 점 : 육사삼거리 중간지점 : 화랑대사거리 → 화랑대...,http://parks.seoul.go.kr/file/info/view.do?fId...,NaN,노원구 공릉2동 산 82-2,중부공원녹지사업소,02-2289-4012,127.076482,37.627077,127.076482,37.627077,NaN
130,131,율현공원,? 율현공원은 보금자리주택 세곡2지구 사업의 결과물로 만들어졌다. 공원은 주택지구의...,157535㎡,2016.11.1,관리시설 : 사무실 조경시설 : 정자(4개) 조형쉼터(10개) 조형파고라(3개) 수...,수목 : 소나무 이팝나무 등 110종 143069주 초화 : 맥문동 벌개미취 등...,http://parks.seoul.go.kr/upload/seditorMulti/2...,NaN,공원 인근에 서울공항 활주로가 위치하고 있어 공원 내에서 드론을 날리거나 헬륨가스 ...,http://parks.seoul.go.kr/file/info/view.do?fId...,강남구,서울특별시 강남구 율현동 밤고개로24길(율현동 56),동부공원녹지사업소,02-459-9452,NaN,NaN,127.115594,37.472332,NaN
131,132,서울로7017,서울로7017은 70년대 준공된 서울역 고가가 안전에 문제점이 도출되면서 철거가 될...,NaN,NaN,NaN,NaN,NaN,중림 만리동(서울로 종점부에서 진입) 지하철- 충정로(경기대입구) - 2 5호선 :...,NaN,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울로7017,서울로운영단,02-313-7017,NaN,NaN,NaN,NaN,NaN


- 1. 공원 면적 정리

In [187]:
park_df.면적.fillna('0', inplace=True)

In [188]:
park_df.면적.isnull().sum()

0

In [189]:
park_df.isnull().sum()

연번                0
공원명               0
공원개요              1
면적                0
개원일              10
주요시설              8
주요식물             44
안내도              30
오시는길             17
이용시참고사항          12
이미지               0
지역                1
공원주소              0
관리부서              0
전화번호              0
X좌표(GRS80TM)     10
Y좌표(GRS80TM)     10
X좌표(WGS84)        1
Y좌표(WGS84)        1
바로가기            114
dtype: int64

In [190]:
area_list = []
for area in park_df.면적:
    area = float(re.sub('[^0-9. ]', '', area).split()[0])
    area_list.append(int(area))
park_df['area'] = area_list

# 강사님 풀이
'''area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))
area = area.apply(lambda x: int(np.round(x)))'''

"area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])\narea[(area == 'nan') | (area == '휴양')] = '0㎡'\narea = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))\narea = area.apply(lambda x: int(np.round(x)))"

- 2. 공원의 면적에 따라 분류, 크기

In [191]:
bins = [-1, 100000, 1000000, max(park_df.area)+1]
labels1 = ['소형', '중형', '대형']
labels2 = [3, 7, 15]
park_df['분류'] = pd.cut(park_df.area, bins=bins, labels=labels1)
park_df['크기'] = pd.cut(park_df.area, bins=bins, labels=labels2)
park_df.head()

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기,area,분류,크기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do,2896887,대형,15
1,2,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do,80683,소형,3
2,3,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019847,37.426449,http://grandpark.seoul.go.kr/,9132690,대형,15
3,4,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432,449290.726,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...,480994,중형,7
4,5,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070,451598.831,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...,2284085,대형,15


- 3. 새로운 데이터 프레임 만들고 저장

In [192]:
df = park_df[['공원명', '지역', 'Y좌표(WGS84)', 'X좌표(WGS84)', 'area', '분류', '크기']]
df.columns = ['공원명', '지역', '위도', '경도', '면적', '분류', '크기']

In [193]:
df.head()

,공원명,지역,위도,경도,면적,분류,크기
0,남산도시자연공원,중구,37.550140,126.990377,2896887,대형,15
1,길동생태공원,강동구,37.540394,127.154779,80683,소형,3
2,서울대공원,과천시,37.426449,127.019847,9132690,대형,15
3,서울숲,성동구,37.543072,127.041798,480994,중형,7
4,월드컵공원,마포구,37.571805,126.878907,2284085,대형,15


In [194]:
df.isnull().sum()

공원명    0
지역     1
위도     1
경도     1
면적     0
분류     0
크기     0
dtype: int64

In [195]:
df[df.경도.isna()]

,공원명,지역,위도,경도,면적,분류,크기
131,서울로7017,중구,NaN,NaN,0,소형,3


In [196]:
df = df.dropna()

In [197]:
df.to_csv('서울공원요약.csv', index=False)

- 4. 공원  시각화

In [200]:
location = [37.541, 126.984]
map = folium.Map(location=location, zoom_start=11)

for i in df.index:
    folium.CircleMarker(
        radius=int(df.크기[i]),
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.분류[i], max_width=200),
        tooltip=f'{df.공원명[i]} {df.면적[i]}㎡',
        color="crimson",
        fill=True
    ).add_to(map)
title = '<h3 align="center" style="font-size:20px">서울시 공원 위치와 크기</h3>'
map.get_root().html.add_child(folium.Element(title))
map

In [199]:
location = [37.541, 126.984]
map = folium.Map(location=location, zoom_start=11, tiles='Stamen Toner')

for i in df.index:
    folium.CircleMarker(
        radius=int(df.크기[i]),
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.분류[i], max_width=200),
        tooltip=f'{df.공원명[i]} {df.면적[i]}㎡',
        color='green',
        fill=True,
        fill_color='green'
    ).add_to(map)
title = '<h3 align="center" style="font-size:20px">서울시 공원 위치와 크기</h3>'
map.get_root().html.add_child(folium.Element(title))
map